In [ ]:
import csv


def get_lists_platforms_and_series():
    
    rows = []
        
    with open('geo_samples.tsv','r') as tsvin, open('series_vs_platforms_kind.csv', 'w') as csvout:
        tsvin = csv.reader(tsvin, delimiter='\t')
        csvout = csv.writer(csvout, delimiter='\t')
        indexes = []
        
        for (i, row) in enumerate(tsvin):
            if i == 0:
                indexes = [row.index(j) for j in ['Series', 'Taxonomy', 'Platform']]
                rows.append(['Series', 'Platforms'])
            
            if row[indexes[1]].lower() in {"homo sapiens", "mus musculus", "rattus norvegicus"}:
                print(i)
                ser = row[indexes[0]].split(";")
                for s in ser: 
                    r = [s, row[indexes[2]], row[indexes[1]].lower()]
                    if r not in rows:
                        rows.append(r)
                        
        print("Length rows: ", len(rows))
        csvout.writerows(rows)
                        

get_lists_platforms_and_series()

In [88]:
import pandas as pd

t_kind = pd.read_csv('series_vs_platforms_kind.csv', sep='\t')


t_used = pd.read_csv('../some_documents/series_vs_gpl_tsv_with_kind.csv', sep='\t')


In [89]:
e = t_kind.Series_taxonomy[t_kind['Series']=='GSE16716']
if (not e.empty):
    print(len(e.index))
    print(e.index[1])
    
e[2045]

4
2045


'homo sapiens'

In [ ]:
new_rows = []
count_many = 0
count_one = 0

for index, row in t_used.iterrows(): 
    kind = t_kind.Series_taxonomy[(t_kind['Series'] == row['Series']) & (t_kind['Platforms'] == row['Platforms'])]
    
    if not kind.empty:
        
        if len(kind.index) == 1:
            new_row = [row['Series'], row['Series_path'], row['Platforms'], row['Platforms_path'], \
                          row['GPL'], row['Series_title']]
            if kind.values[0] != row['Series_taxonomy']:
                count_one = count_one + 1
                print(index, " ", row['Series'], " ", row['Platforms'], " ", row['Series_taxonomy'], " ", kind.values[0])
                new_row.append(kind.values[0])
            else:
                new_row.append(row['Series_taxonomy'])
                
            new_rows.append(new_row)
        else:
            for i in range(0,len(kind.index),1):
                new_row = [row['Series'], row['Series_path'], row['Platforms'], row['Platforms_path'], \
                          row['GPL'], row['Series_title'], kind[kind.index[i]]]
                new_rows.append(new_row)
            count_many = count_many + 1
    else:
        new_row = [row['Series'], row['Series_path'], row['Platforms'], row['Platforms_path'], \
                          row['GPL'], row['Series_title'], row['Series_taxonomy']]
        new_rows.append(new_row)

print("count_one: ", count_one)
print("count_many: ", count_many)
df = pd.DataFrame(new_rows,columns=['Series','Series_path','Platforms','Platforms_path','GPL','Series_title',\
                               'Series_taxonomy'])
print("drop")
df_clear = df.drop_duplicates(subset=['Series', 'Platforms', 'Series_taxonomy'], keep=False)


In [99]:
df.index

RangeIndex(start=0, stop=74245, step=1)

In [102]:
df_clear.to_csv('../some_documents/series_vs_gpl_tsv_with_kind_2.tsv', sep='\t', index=False)